# TCP

<img src='res/tcp01.png' width=500px />
<img src='res/tcp02.png' width=500px />

## 缩写

congestion control algorithm(CCA), congestion window(CWND), receiver's advertised window(RWND), maximum segment size(MSS), slow start threshold(ssthresh), round-trip delay/time(RTD,RTT), Retransmission Timeout(RTO)

--------





## slow start,congestion control,fast retransmit,fast recovery

<img src='res/tcp03.png' width=500px />

[RFC2001](res/rfc2001.pdf)

<img src='res/tcp04.png' width=500px />

--------

## tcp receive slow path / fast path
### net/ipv4/tcp_input.c


```c

/*
 *	TCP receive function for the ESTABLISHED state. 
 *
 *	It is split into a fast path and a slow path. The fast path is 
 * 	disabled when:
 *	- A zero window was announced from us - zero window probing
 *        is only handled properly in the slow path. 
 *	- Out of order segments arrived.
 *	- Urgent data is expected.
 *	- There is no buffer space left
 *	- Unexpected TCP flags/window values/header lengths are received
 *	  (detected by checking the TCP header against pred_flags) 
 *	- Data is sent in both directions. Fast path only supports pure senders
 *	  or pure receivers (this means either the sequence number or the ack
 *	  value must stay constant)
 *	- Unexpected TCP option.
 *
 *	When these conditions are not satisfied it drops into a standard 
 *	receive procedure patterned after RFC793 to handle all cases.
 *	The first three cases are guaranteed by proper pred_flags setting,
 *	the rest is checked inline. Fast processing is turned on in 
 *	tcp_data_queue when everything is OK.
 */
int tcp_rcv_established(struct sock *sk, struct sk_buff *skb,
			struct tcphdr *th, unsigned len)
{
	struct tcp_sock *tp = tcp_sk(sk);

	/*
	 *	Header prediction.
	 *	The code loosely follows the one in the famous 
	 *	"30 instruction TCP receive" Van Jacobson mail.
	 *	
	 *	Van's trick is to deposit buffers into socket queue 
	 *	on a device interrupt, to call tcp_recv function
	 *	on the receive process context and checksum and copy
	 *	the buffer to user space. smart...
	 *
	 *	Our current scheme is not silly either but we take the 
	 *	extra cost of the net_bh soft interrupt processing...
	 *	We do checksum and copy also but from device to kernel.
	 */

	tp->rx_opt.saw_tstamp = 0;

	/*	pred_flags is 0xS?10 << 16 + snd_wnd
	 *	if header_predition is to be made
	 *	'S' will always be tp->tcp_header_len >> 2
	 *	'?' will be 0 for the fast path, otherwise pred_flags is 0 to
	 *  turn it off	(when there are holes in the receive 
	 *	 space for instance)
	 *	PSH flag is ignored.
	 */

	if ((tcp_flag_word(th) & TCP_HP_BITS) == tp->pred_flags &&
		TCP_SKB_CB(skb)->seq == tp->rcv_nxt) {
		int tcp_header_len = tp->tcp_header_len;
```

1. 为了提高处理速度，提供了快慢两条路径。大部分都是预期以内的，比如没有窗口大小调整，flag有ack，header len是固定的，也就是没有额外新的option，着一些就对应着tcp header的第3个32bit字段。这个称之为pred_flags，预测的。如果预测的和实际的相同，同时受到的seq num == 下一个rec num，就进入快速通道

2. `tcp_flag_word` 就是取tcp header中第三个32bit的值。内核中用了个技巧，用个union把tcphdr 和 tcp_word_hdr（5个32bit的值=20bytes）统一了，方便取第3个32bit的值

```c
struct tcphdr {
	__be16	source;
	__be16	dest;
	__be32	seq;
	__be32	ack_seq;
#if defined(__LITTLE_ENDIAN_BITFIELD)
	__u16	res1:4,
		doff:4,
		fin:1,
		syn:1,
		rst:1,
		psh:1,
		ack:1,
		urg:1,
		ece:1,
		cwr:1;
#elif defined(__BIG_ENDIAN_BITFIELD)
	__u16	doff:4,
		res1:4,
		cwr:1,
		ece:1,
		urg:1,
		ack:1,
		psh:1,
		rst:1,
		syn:1,
		fin:1;
#else
#error	"Adjust your <asm/byteorder.h> defines"
#endif
	__be16	window;
	__sum16	check;
	__be16	urg_ptr;
};

/*
 *	The union cast uses a gcc extension to avoid aliasing problems
 *  (union is compatible to any of its members)
 *  This means this part of the code is -fstrict-aliasing safe now.
 */
union tcp_word_hdr {
	struct tcphdr hdr;
	__be32        words[5];
};

#define tcp_flag_word(tp) (((union tcp_word_hdr *)(tp))->words[3])
```

----------

-------
-------

## linux2.6/include/net/tcp.h

```c
/*
 * The next routines deal with comparing 32 bit unsigned ints
 * and worry about wraparound (automatic with unsigned arithmetic).
 */

static inline int before(__u32 seq1, __u32 seq2)
{
        return (__s32)(seq1-seq2) < 0;
}

static inline int after(__u32 seq1, __u32 seq2)
{
	return (__s32)(seq2-seq1) < 0;
}


/* is s2<=s1<=s3 ? */
static inline int between(__u32 seq1, __u32 seq2, __u32 seq3)
{
	return seq3 - seq2 >= seq1 - seq2;
}
```

1. 比较tricky的比较两个无符号sequence number的办法。主要的问题是seq num可能会wrap around，比如4bits的值，seq1是1111,seq2是0，seq1 应该小于 seq2，如果直接比较是不对的

2. before函数为例， seq1-seq2 = seq1 + seq2的补码 = seq1 + （(1<<32)-seq2）= seq1-seq2 + (1<<32)，如果没有发生wraparound，值是对的。如果发生wraparound，seq1-seq2就是两个数之间的距离，只要(距离>=1^31)，那么符号位就是1，就是个负数，符号也是对的。所以这个函数有效，必须wrapround发生的距离不太大，即seq1,seq2的差值>=1^31。实际seq num是线性递增的，1^31是个很大的数，一般不会超过

3. between函数，与before类似，两个数相减计算的永远是这两个数之间的距离(正数就是距离，负数就是补码距离=1^32+负数)，无论是否wraparound。所以只要3到2的距离大于1到2的距离，那么1就在2和3之间

Very nice and tricky method !!!

-------------------------